## setup data path final norm

In [1]:
import numpy as np
import os
import pandas as pd
import pprint
import csv
import keras
#path constants
train_path = '../../../iterations/data/final/train_norm'
test_path = '../../../iterations/data/final/test_norm'

cleaned_train_path = '../../../cleaned_data/train'
cleaned_test_path = '../../../cleaned_data/test'
#type constants
vehicle_types = ['ZVe44', 'ZV573', 'ZV63d', 'ZVfd4', 'ZVa9c', 'ZVa78', 'ZV252']

#two label dataframes
train_label_df = pd.read_csv('../../../iterations/data/final' + '/label.csv', delimiter = ',', encoding = 'utf-8')
test_label_df = pd.read_csv('../../../iterations/data/final' + '/label.csv', delimiter = ',', encoding = 'utf-8')

Using TensorFlow backend.


## decide length of the input

In [39]:
def getLabel(filename, label_df):
    idx = label_df.loc[label_df['sample_file_name'] == filename]
    return idx.iloc[0]['label']

def cal_length(path, vehicle_type, label_df):

#vehicle_type: one string element under vehicle_types = ['ZVe44', 'ZV573', 'ZV63d', 'ZVfd4', 'ZVa9c', 'ZVa78', 'ZV252']
    path = path + '/' + vehicle_type
    #these are variables to calculate traversing progress (DO NOT CHANGE)
    counts_per_percent = int(len(os.listdir(path)) / 100)
    percentage_completion = 0
    counter = 0
    
    single_len=0
    file_count=0
    file_len=0
    for file in os.listdir(path):
        sample_df = pd.read_csv(path + '/' + file, delimiter = ',', encoding = 'utf-8')
        file_len+=len(sample_df)
        file_count+=1
        if len(sample_df)>=single_len:
            single_len=len(sample_df)
    ave_file_len=file_len/file_count
    print("AVG Length: ",ave_file_len,' file count: ',file_count,' max_len: ',single_len)

In [40]:
cal_length(train_path,'ZVfd4',train_label_df)

AVG Length:  129.54157782515992  file count:  938  max_len:  392


In [41]:
file_len_weighted=round(392*0.7)
file_len_weighted

274

## load the data

In [2]:
def getLabel(filename, label_df):
    idx = label_df.loc[label_df['sample_file_name'] == filename]
    return idx.iloc[0]['label']

def TraverseFiles(path, vehicle_type, label_df, length):

#vehicle_type: one string element under vehicle_types = ['ZVe44', 'ZV573', 'ZV63d', 'ZVfd4', 'ZVa9c', 'ZVa78', 'ZV252']
    path = path + '/' + vehicle_type
    #these are variables to calculate traversing progress (DO NOT CHANGE)
    counts_per_percent = int(len(os.listdir(path)) / 100)
    percentage_completion = 0
    counter = 0
    
    lables=np.array([])
    file_list= []
    #file_len=file_len_weighted
    for file in os.listdir(path):
        sample_df = pd.read_csv(path + '/' + file, delimiter = ',', encoding = 'utf-8')
        if len(sample_df)==0:
            continue
        elif len(sample_df)!=0:
            sample_array=np.array(sample_df.iloc[:,0:11])
            if len(sample_array)<length:
                n_zeros=length-len(sample_array)
                sample_array=np.concatenate((np.zeros((n_zeros,11)),sample_array))
                #print(len(sample_array))
            elif len(sample_array)>length:
                sample_array=sample_array[0:length]
                #print(len(sample_array))
            elif len(sample_array)==length:
                sample_array=sample_array
                #print(len(sample_array))
            #np.float(sample_array)
            #sample_array=torch.from_numpy(sample_array).float()
            #file_list=torch.cat((file_list,sample_array[0:76,]), dim=1)
        file_list.append(sample_array)
        l=np.array(label_df.loc[label_df['sample_file_name'] == file])
        lables=np.append(lables,[l[:,1]])
        #belows are to show traversing progress (DO NOT CHANGE)
        counter += 1
        if counter == counts_per_percent:
            counter = 0
            percentage_completion += 1
            print('traversing files under', path, ':', percentage_completion, "%", end="\r", flush=True)

    return file_list,lables

In [43]:
ZVfd4_array_train, ZVfd4_labels_train = TraverseFiles(train_path,'ZVfd4',train_label_df,file_len_weighted)
ZVfd4_array_test, ZVfd4_labels_test = TraverseFiles(test_path,'ZVfd4',test_label_df,file_len_weighted)

In [46]:
ZVfd4_x_train=np.array(ZVfd4_array_train)
ZVfd4_labels_train=np.array(ZVfd4_labels_train,dtype=int)

In [56]:
ZVfd4_x_test=np.array(ZVfd4_array_test)
ZVfd4_labels_test=np.array(ZVfd4_labels_test,dtype=int)

In [57]:
print(ZVfd4_x_train.shape)
print(ZVfd4_labels_train.shape)
print(ZVfd4_x_test.shape)
print(ZVfd4_labels_test.shape)

(938, 274, 11)
(938,)
(235, 274, 11)
(235,)


## build and train model

In [58]:
keras.backend.clear_session()
lstm = keras.Sequential()
lstm.add(keras.layers.Dropout(0.2))
lstm.add(keras.layers.LSTM(16,dropout=0.2, recurrent_dropout=0.2,return_sequences=True))
lstm.add(keras.layers.BatchNormalization())
lstm.add(keras.layers.LSTM(16,dropout=0.2, recurrent_dropout=0.2))
lstm.add(keras.layers.BatchNormalization())
lstm.add(keras.layers.Dense(1, activation = 'sigmoid'))

lstm.compile(
    loss='binary_crossentropy',
    optimizer='Adam',
    metrics=['accuracy']
)

In [59]:
lstm.fit(ZVfd4_x_train,
    ZVfd4_labels_train,
    epochs = 1000,
    batch_size=200
)

Epoch 1/1000
938/938 [==============================] - 3s 3ms/step - loss: 0.8116 - accuracy: 0.5224
Epoch 2/1000
938/938 [==============================] - 1s 1ms/step - loss: 0.7724 - accuracy: 0.5288
Epoch 3/1000
938/938 [==============================] - 1s 1ms/step - loss: 0.7558 - accuracy: 0.5117
Epoch 4/1000
938/938 [==============================] - 1s 1ms/step - loss: 0.7590 - accuracy: 0.5075
Epoch 5/1000
938/938 [==============================] - 1s 2ms/step - loss: 0.7314 - accuracy: 0.5544
Epoch 6/1000
938/938 [==============================] - 1s 1ms/step - loss: 0.7276 - accuracy: 0.5267
Epoch 7/1000
938/938 [==============================] - 1s 1ms/step - loss: 0.7270 - accuracy: 0.5341
Epoch 8/1000
938/938 [==============================] - 1s 1ms/step - loss: 0.7088 - accuracy: 0.5576
Epoch 9/1000
938/938 [==============================] - 1s 1ms/step - loss: 0.7087 - accuracy: 0.5704
Epoch 10/1000
938/938 [==============================] - 1s 1ms/step - loss: 0.717

938/938 [==============================] - 1s 2ms/step - loss: 0.6438 - accuracy: 0.6450
Epoch 160/1000
938/938 [==============================] - 1s 2ms/step - loss: 0.6487 - accuracy: 0.6311
Epoch 161/1000
938/938 [==============================] - 1s 2ms/step - loss: 0.6362 - accuracy: 0.6418
Epoch 162/1000
938/938 [==============================] - 1s 2ms/step - loss: 0.6550 - accuracy: 0.6247
Epoch 163/1000
938/938 [==============================] - 1s 2ms/step - loss: 0.6582 - accuracy: 0.6130
Epoch 164/1000
938/938 [==============================] - 1s 2ms/step - loss: 0.6497 - accuracy: 0.6247
Epoch 165/1000
938/938 [==============================] - 1s 2ms/step - loss: 0.6482 - accuracy: 0.6407
Epoch 166/1000
938/938 [==============================] - 1s 2ms/step - loss: 0.6480 - accuracy: 0.6258
Epoch 167/1000
938/938 [==============================] - 1s 2ms/step - loss: 0.6491 - accuracy: 0.6130
Epoch 168/1000
938/938 [==============================] - 1s 2ms/step - loss: 0

938/938 [==============================] - 2s 2ms/step - loss: 0.6362 - accuracy: 0.6322
Epoch 316/1000
938/938 [==============================] - 1s 2ms/step - loss: 0.6243 - accuracy: 0.6461
Epoch 317/1000
938/938 [==============================] - 1s 2ms/step - loss: 0.6341 - accuracy: 0.6333
Epoch 318/1000
938/938 [==============================] - 2s 2ms/step - loss: 0.6400 - accuracy: 0.6311
Epoch 319/1000
938/938 [==============================] - 1s 2ms/step - loss: 0.6258 - accuracy: 0.6514
Epoch 320/1000
938/938 [==============================] - 1s 2ms/step - loss: 0.6257 - accuracy: 0.6322
Epoch 321/1000
938/938 [==============================] - 1s 2ms/step - loss: 0.6445 - accuracy: 0.6141
Epoch 322/1000
938/938 [==============================] - 1s 2ms/step - loss: 0.6166 - accuracy: 0.6588
Epoch 323/1000
938/938 [==============================] - 2s 2ms/step - loss: 0.6327 - accuracy: 0.6343
Epoch 324/1000
938/938 [==============================] - 1s 2ms/step - loss: 0

938/938 [==============================] - 1s 2ms/step - loss: 0.6151 - accuracy: 0.6397
Epoch 472/1000
938/938 [==============================] - 1s 2ms/step - loss: 0.6214 - accuracy: 0.6546
Epoch 473/1000
938/938 [==============================] - 1s 2ms/step - loss: 0.6138 - accuracy: 0.6471
Epoch 474/1000
938/938 [==============================] - 1s 2ms/step - loss: 0.6404 - accuracy: 0.6343
Epoch 475/1000
938/938 [==============================] - 1s 2ms/step - loss: 0.6218 - accuracy: 0.6493
Epoch 476/1000
938/938 [==============================] - 1s 2ms/step - loss: 0.6291 - accuracy: 0.6514
Epoch 477/1000
938/938 [==============================] - 1s 2ms/step - loss: 0.6228 - accuracy: 0.6535
Epoch 478/1000
938/938 [==============================] - 1s 2ms/step - loss: 0.6205 - accuracy: 0.6503
Epoch 479/1000
938/938 [==============================] - 1s 2ms/step - loss: 0.6207 - accuracy: 0.6695
Epoch 480/1000
938/938 [==============================] - 1s 2ms/step - loss: 0

938/938 [==============================] - 1s 1ms/step - loss: 0.6360 - accuracy: 0.6407
Epoch 628/1000
938/938 [==============================] - 1s 1ms/step - loss: 0.6284 - accuracy: 0.6482
Epoch 629/1000
938/938 [==============================] - 1s 1ms/step - loss: 0.6075 - accuracy: 0.6642
Epoch 630/1000
938/938 [==============================] - 1s 1ms/step - loss: 0.6100 - accuracy: 0.6780
Epoch 631/1000
938/938 [==============================] - 1s 1ms/step - loss: 0.6206 - accuracy: 0.6493
Epoch 632/1000
938/938 [==============================] - 1s 1ms/step - loss: 0.6251 - accuracy: 0.6557
Epoch 633/1000
938/938 [==============================] - 1s 1ms/step - loss: 0.6162 - accuracy: 0.6610
Epoch 634/1000
938/938 [==============================] - 1s 1ms/step - loss: 0.6227 - accuracy: 0.6375
Epoch 635/1000
938/938 [==============================] - 1s 1ms/step - loss: 0.6312 - accuracy: 0.6525
Epoch 636/1000
938/938 [==============================] - 1s 1ms/step - loss: 0

938/938 [==============================] - 1s 1ms/step - loss: 0.6048 - accuracy: 0.6620
Epoch 784/1000
938/938 [==============================] - 1s 1ms/step - loss: 0.6114 - accuracy: 0.6706
Epoch 785/1000
938/938 [==============================] - 1s 1ms/step - loss: 0.6204 - accuracy: 0.6514
Epoch 786/1000
938/938 [==============================] - 1s 1ms/step - loss: 0.6336 - accuracy: 0.6471
Epoch 787/1000
938/938 [==============================] - 1s 1ms/step - loss: 0.6075 - accuracy: 0.6791
Epoch 788/1000
938/938 [==============================] - 1s 1ms/step - loss: 0.5955 - accuracy: 0.6930
Epoch 789/1000
938/938 [==============================] - 1s 1ms/step - loss: 0.6201 - accuracy: 0.6557
Epoch 790/1000
938/938 [==============================] - 1s 1ms/step - loss: 0.6086 - accuracy: 0.6695
Epoch 791/1000
938/938 [==============================] - 1s 1ms/step - loss: 0.6245 - accuracy: 0.6418
Epoch 792/1000
938/938 [==============================] - 1s 1ms/step - loss: 0

938/938 [==============================] - 1s 1ms/step - loss: 0.6157 - accuracy: 0.6567
Epoch 940/1000
938/938 [==============================] - 1s 1ms/step - loss: 0.6161 - accuracy: 0.6567
Epoch 941/1000
938/938 [==============================] - 1s 1ms/step - loss: 0.6105 - accuracy: 0.6663
Epoch 942/1000
938/938 [==============================] - 1s 1ms/step - loss: 0.6220 - accuracy: 0.6535
Epoch 943/1000
938/938 [==============================] - 1s 1ms/step - loss: 0.6037 - accuracy: 0.6780
Epoch 944/1000
938/938 [==============================] - 1s 1ms/step - loss: 0.6010 - accuracy: 0.6780
Epoch 945/1000
938/938 [==============================] - 1s 1ms/step - loss: 0.6262 - accuracy: 0.6535
Epoch 946/1000
938/938 [==============================] - 1s 1ms/step - loss: 0.6142 - accuracy: 0.6599
Epoch 947/1000
938/938 [==============================] - 1s 1ms/step - loss: 0.6163 - accuracy: 0.6588
Epoch 948/1000
938/938 [==============================] - 1s 1ms/step - loss: 0

In [60]:
lstm.save('lstm_model_ZVfd4.h5')

## acc on the splited test set

In [61]:
ZVfd4_pred=lstm.predict_classes(ZVfd4_x_test)

In [62]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
print(classification_report(ZVfd4_labels_test, ZVfd4_pred,digits=4))
print(accuracy_score(ZVfd4_labels_test, ZVfd4_pred))

              precision    recall  f1-score   support

           0     0.6948    0.7754    0.7329       138
           1     0.6173    0.5155    0.5618        97

   micro avg     0.6681    0.6681    0.6681       235
   macro avg     0.6560    0.6454    0.6473       235
weighted avg     0.6628    0.6681    0.6623       235

0.6680851063829787


## acc on cleaned test data

In [4]:
train_path = '../../../data/train'
test_path = '../../../data/test'

cleaned_train_path = '../../../cleaned_data/train'
cleaned_test_path = '../../../cleaned_data/test'
#type constants
vehicle_types = ['ZVe44', 'ZV573', 'ZV63d', 'ZVfd4', 'ZVa9c', 'ZVa78', 'ZV252']

#two label dataframes
train_label_df = pd.read_csv(cleaned_train_path + '/train_label.csv', delimiter = ',', encoding = 'utf-8')
test_label_df = pd.read_csv(cleaned_test_path + '/test_label.csv', delimiter = ',', encoding = 'utf-8')

In [12]:
ZVfd4_array_test, ZVfd4_labels_test = TraverseFiles(test_path,'ZVfd4',test_label_df,274)

In [7]:
model = keras.models.load_model('lstm_model_ZVfd4.h5')

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.


In [8]:
ZVfd4_cleaned_x=np.array(ZVfd4_array_test)
ZVfd4_cleaned_labels=np.array(ZVfd4_labels_test,dtype=int)

In [10]:
pred_cleaned=model.predict_classes(ZVfd4_cleaned_x)

In [11]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
print(classification_report(ZVfd4_cleaned_labels, pred_cleaned,digits=4))
print(accuracy_score(ZVfd4_cleaned_labels, pred_cleaned))

              precision    recall  f1-score   support

           0     0.6847    0.5869    0.6320       259
           1     0.5579    0.6585    0.6040       205

   micro avg     0.6185    0.6185    0.6185       464
   macro avg     0.6213    0.6227    0.6180       464
weighted avg     0.6286    0.6185    0.6197       464

0.6185344827586207
